In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1047b6f0-9ad1-4546-b102-24868afc482c;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser

In [5]:
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [f.col(nc+'.'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [6]:
topic = 'bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO'
landing_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido/'
url = 'http://schema-registry-cp-schema-registry.ingestion.svc:8081/subjects/' + topic + '-value/versions/latest'

In [7]:
# Get Schema from schema registry
response = urlopen(url)
data_json = json.loads(response.read())

URLError: <urlopen error [Errno -2] Name or service not known>

In [8]:
# Reading last bronze data
try:
    df_bronze = (
        spark
        .read
        .format("delta")
        .load(bronze_path)
    )

    list_result = (
        df_bronze
        .agg({"timestamp_kafka": "max"}).collect()
    )
    
    snapshot_flag = False
    last_date = list_result[0].asDict()['max(timestamp_kafka)']
    
except:
    snapshot_flag = True
    last_date = parser.parse("01-01-2021 12:00AM")

In [9]:
print(snapshot_flag, last_date)

True 2021-01-01 00:00:00


In [10]:
# Reading landing data
df_table = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp")>last_date)
    .withColumnRenamed("timestamp", "timestamp_kafka")
)

In [11]:
df_table.count()

311856

In [12]:
from_avro_options= {"mode":"PERMISSIVE"}
df_temp = (
    df_table
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

In [13]:
df_table_flat = flatten_df(df_temp)

In [14]:
if snapshot_flag:
    
    landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/hr_omni_pedido_snapshot/'
    
    # Load schema from schema registry
    topic_snapshot = 'temp_bnsvher602.linx_db.dbo.HR_OMNI_PEDIDO'
    url_snapshot = 'http://schema-registry-cp-schema-registry.ingestion.svc:8081/subjects/' + topic_snapshot + '-value/versions/latest'
    response_snapshot = urlopen(url_snapshot)
    data_json_snapshot = json.loads(response_snapshot.read())
    
    # Reading landing data
    df_table_snapshot = (
        spark
        .read
        .format("avro")
        .load(landing_path_snapshot)
        .withColumnRenamed("timestamp", "timestamp_kafka")
    )
    
    df_temp_snapshot = (
        df_table_snapshot
        .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
        .select(from_avro("fixedValue", data_json_snapshot['schema']), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
    )
    
    df_table_flat_snapshot = flatten_df(df_temp_snapshot)
    
    df_table_flat = df_table_flat.union(df_table_flat_snapshot)

In [15]:
df_table_flat.count()

5276558

In [16]:
(
    df_table_flat
    .write
    .partitionBy("year", "month", "day", "hour")
    .format("delta")
    .mode("append")
    .option("checkpointLocation", bronze_path + "checkpoint")
    .save(bronze_path)
)

In [21]:
##################################
### Verificar está funcionando ###
##################################

In [16]:
count_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .count()
)

print (count_bronze)

261660


In [18]:
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/produtos_snapshot/'

count_landing = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .count()
)

count_landing_snap = (
    spark
    .read
    .format("avro")
    .load(landing_path_snapshot)
    .count()
)


print (count_landing + count_landing_snap)

261660
